# The Historically Best Players in League of Legends by Role

### Top - Knight

### Jungle - Kanavi

### Mid - Knight

### ADC - Rekkles

### Support - Spirit

There you have it, backed up by unbiased and objective mathematical computations.  The greatest players in League of Legends by role.  If you like what you see, I wouldn't worry about reading the rest of this.  Just say you saw someone prove your opinion and cite this. [<sup>1</sup>](#fn1)

If you take umbridge with these results, please read on and try to figure out where and how your analysis differs from mine.

<span id = "fn1"><sup>1</sup> Please don't do this.</span>

## Disclaimer

The statistics I used as the basis for this analysis were downloaded from rotowire.com on 2021-07-29.  I used stats for NA LCS, EU LCS, LPL, and LCK.  

This work isn't endorsed by Riot Games and doesn't reflect the views or opinions of Riot Games or anyone officially involved in producing or managing Riot Games properties.  Riot Games, and all associated properties are trademarks or registered trademarks or Riot Games, Inc.

## Overview

So, what's my plan with all these stats.  How will I turn this objective data into objective facts about the best players to ever touch the game of League of Legends?  By applying some basics of data science that I recently learned.  

The first step is always to load the data and clean it up a bit to suit our purpose.  I'll keep all the grunt work hidden and just present the executive summary of what I'm doing and why so you don't have to scroll through countless mistakes and explorations that end up going nowhere.

After we've loaded and cleaned up the data, our next (and final) step is to hit the data with some math until it gives up its tasty secrets.  

Are you ready?

## The Appetizer

Okay, first we have the preliminaries.  Mainly importing the proper libraries and loading up the stats.  Here we go!

In [1]:
import pandas as pd

In [2]:
nalcs_data = pd.read_csv("lol-player-stats_NA.csv", header=1)
lpl_data = pd.read_csv("lol-player-stats_CN.csv", header=1)
eulcs_data = pd.read_csv("lol-player-stats_EU.csv", header=1)
lck_data = pd.read_csv("lol-player-stats_KR.csv", header=1)

nalcs_data.head()

,Player,Team,POS,M,G,W,L,K,D,A,...,KP%,KS%,CS,CSM,GLD%,DPM,FB,DTH%,WPM,WCPM
0,Jensen,Team Liquid,MID,132,176,114,62,610,291,976,...,68.8,26.5,53337,9.0,22.7,508.2,13,16.6,0.37,0.19
1,Zven,Cloud9,ADC,85,119,85,34,511,173,687,...,65.8,28.1,35037,9.3,23.7,525.5,14,14.8,0.45,0.37
2,Blaber,Cloud9,JNG,100,134,93,41,476,320,990,...,72.4,23.5,28183,6.6,20.0,360.1,32,23.8,0.43,0.37
3,Bjergsen,Team SoloMid,MID,85,130,75,55,429,189,683,...,70.5,27.2,40126,8.7,22.8,501.7,12,13.7,0.42,0.16
4,Tactical,Team Liquid,ADC,77,107,69,38,429,237,553,...,68.4,29.9,33979,9.4,24.1,557.5,11,21.2,0.52,0.35


## The Main course

First look at the data and it seems pretty legit.  From the rotowire website, here is a list of what each of the columns represent

|Label | Description |
| -: | :- |
| POS | Position |
| M | Matches played |
| G | Games played |
| W | Games won |
| L | Games lost |
| K | Kills |
| D | Deaths |
| A | Assists |
| KDA | Kill/Death/Assist Ratio ((K + A)/D) |
| KP% | Kill Participation ((Kills + Assists) / Total # of kills by team) |
| KS% | Kill Share (Kills / Total # of kills by team) |
| CS | Creep Score |
| CSM | Creep Score per Minute |
| GLD% | Gold Share (Percentage of team's total gold) |
| DPM | Damage per Minute |
| FB | First Blood kills |
| WPM | Wards Placed per Minute |
| WCPM | Wards Cleared per Minute |


Right off the bat, I don't care about the team of each player so I'm going to drop that column.  Other than that, it's time to staple everything together and look at the full picture.

In [3]:
alldata = nalcs_data.append(lpl_data.append(eulcs_data.append(lck_data, ignore_index=True), ignore_index=True), ignore_index=True)

alldata = alldata.drop(labels={'Team'}, axis=1)

alldata.tail()

,Player,POS,M,G,W,L,K,D,A,KDA,KP%,KS%,CS,CSM,GLD%,DPM,FB,DTH%,WPM,WCPM
1449,Kuzan,MID,1,1,0,1,0,2,4,2.0,40.0,0.0,320,8.3,19.3,325.1,0,20.0,0.55,0.13
1450,MapSSi,SUP,1,1,0,1,0,5,12,2.4,63.2,0.0,56,1.5,12.5,301.4,0,23.8,1.77,0.42
1451,Max,SUP,3,4,0,4,0,17,9,0.5,45.0,0.0,98,0.7,12.9,66.2,0,24.3,1.14,0.29
1452,Naehyun,MID,1,1,0,1,0,0,0,0.0,0.0,0.0,265,9.5,22.8,172.4,0,0.0,0.47,0.36
1453,Wizer,TOP,1,1,0,1,0,3,3,1.0,30.0,0.0,358,9.8,21.8,451.7,0,16.7,0.46,0.36


In [4]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  1454 non-null   object 
 1   POS     1454 non-null   object 
 2   M       1454 non-null   int64  
 3   G       1454 non-null   int64  
 4   W       1454 non-null   int64  
 5   L       1454 non-null   int64  
 6   K       1454 non-null   int64  
 7   D       1454 non-null   int64  
 8   A       1454 non-null   int64  
 9   KDA     1454 non-null   float64
 10  KP%     1454 non-null   float64
 11  KS%     1454 non-null   float64
 12  CS      1454 non-null   int64  
 13  CSM     1454 non-null   float64
 14  GLD%    1454 non-null   float64
 15  DPM     1454 non-null   float64
 16  FB      1454 non-null   int64  
 17  DTH%    1454 non-null   float64
 18  WPM     1454 non-null   float64
 19  WCPM    1454 non-null   float64
dtypes: float64(9), int64(9), object(2)
memory usage: 227.3+ KB


In [5]:
alldata.describe()

,M,G,W,L,K,D,A,KDA,KP%,KS%,CS,CSM,GLD%,DPM,FB,DTH%,WPM,WCPM
count,1454.000000,1454.000000,1454.000000,1454.000000,1454.00000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000
mean,27.507565,57.383081,28.737964,28.645117,136.90784,135.997249,326.819120,3.255846,67.117125,19.430812,12842.824622,6.292091,19.692916,348.029642,5.550206,20.230261,0.688301,0.290715
std,28.394495,66.593639,39.093577,30.196841,197.82532,155.611707,428.175048,1.514451,8.339060,9.499799,17777.612792,2.965502,3.502251,154.686356,7.971207,4.204781,0.430202,0.130578
min,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,31.000000,0.300000,8.200000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,12.000000,4.000000,7.000000,19.00000,31.000000,58.000000,2.300000,63.600000,13.300000,1728.500000,4.300000,17.700000,208.975000,1.000000,17.700000,0.400000,0.180000
50%,19.000000,39.000000,16.000000,22.000000,68.00000,97.000000,194.500000,3.000000,68.100000,20.300000,6567.500000,7.600000,20.700000,362.250000,3.000000,20.100000,0.500000,0.280000
75%,38.000000,76.000000,36.000000,38.000000,170.75000,178.750000,424.750000,3.975000,71.700000,26.400000,16485.750000,8.700000,22.300000,472.475000,7.000000,22.700000,0.850000,0.380000
max,181.000000,468.000000,292.000000,214.000000,1811.00000,1246.000000,4158.000000,16.000000,100.000000,66.700000,138085.000000,11.000000,26.600000,770.000000,96.000000,38.900000,2.040000,0.710000


In [6]:
pd.isna(alldata).values.any()

False

We now know some basic information about the data and that there is no missing data.  Good for us, but there's some trouble brewing.  

I mentioned at the start that I downloaded statistics by region, but players move between regions.  Not only that, they also change roles! How rude and inconsiderate.

So, what to do?  Well, we want to make sure that each player's stats are in one location and not spread out over multiple entries to account for play across multiple regions.  We also don't want to mix a player's stats across roles.  An ADC player would be very unhappy indeed if their time playing as a support mattered in an evaluation of how well they performed in the ADC role. 

In [7]:
print("Total number of entries:")
alldata['Player'].count()

Total number of entries:


1454

In [8]:
print("Number of unique players:")
alldata['Player'].nunique()

Number of unique players:


726

In [9]:
print("Number of unique player + postion combinations:")
alldata.groupby(['Player','POS']).sum().reset_index()['Player'].count()

Number of unique player + postion combinations:


780

Our target is to group everything by player + position, so how do we do that?  

The first thing to note is we have two kinds of statistics at our fingertips: Cumulative and averaged.

Cumulative stats are those like Kills, Deaths, or Creep Score.  Straightforward addition will do to combine these stats from multiple entries.  

Averaged stats are those such as KDA or CSM.  These are averages, or rates, that are particular to the games they are calculated for.  Therefore, to properly account for these stats across different entries we will employ a weighted average based on the total number of games played.  As an example, if someone has 90 games at a KDA of 5 and 10 games at a KDA of 7, to get the total KDA across all 100 games we take 0.9 * 5 + 0.1 * 7.

These differences in stat types mean that we will need to handle these two types separately and then recombine them at the end.

In [10]:
cumulativestats = alldata.loc[:,['Player','POS','M','G','W','L','K','D','A','CS','FB']]

averagedstats = alldata.loc[:,['Player','POS','KDA','KP%','KS%', 'CSM', 'GLD%','DPM','DTH%','WPM','WCPM']]

Now comes a slightly messy part.  I want to add a column that lists the total number of games played by Player + POS.  But I need to do this before I combine the information, which means that I need multiple entries to have the same value.  If you know of a cleaner way to do this please let me know.

In [11]:
totalgamesplayed = {}

for i in range(len(averagedstats)):
    totalgamesplayed[i] = cumulativestats.groupby(['Player','POS']).sum().loc[(cumulativestats.loc[i,'Player'],cumulativestats.loc[i,'POS']),'G']

In [12]:
tgp_df = pd.DataFrame(totalgamesplayed, index = list(totalgamesplayed)).T.iloc[:,0]

cumulativestats = cumulativestats.join(tgp_df)
cumulativestats = cumulativestats.rename({cumulativestats.columns[11]:'TotalGames'}, axis='columns')

averagedstats = averagedstats.assign(Weight = cumulativestats['G'] / cumulativestats['TotalGames'])

weightedaveragedstats = averagedstats.iloc[:,0:2]
weightedaveragedstats = weightedaveragedstats.assign(KDA = averagedstats['KDA'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(KPP = averagedstats['KP%'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(KSP = averagedstats['KS%'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(CSM = averagedstats['CSM'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(GLDP = averagedstats['GLD%'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(DPM = averagedstats['DPM'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(DTHP = averagedstats['DTH%'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(WPM = averagedstats['WPM'] * averagedstats['Weight'])
weightedaveragedstats = weightedaveragedstats.assign(WCPM = averagedstats['WCPM'] * averagedstats['Weight'])

weightedaveragedstats.head()

finalstats = cumulativestats.groupby(['Player','POS']).sum().join(weightedaveragedstats.groupby(['Player','POS']).sum())

We're almost to the final stage, just a little more cleaning up to do.  First I'm going to drop two columns (CS and TotalGames) and add two columns (one for First Blood percentage and one for Win percentage).

In [13]:
finalstats = finalstats.drop(labels={'CS','TotalGames'}, axis=1)
finalstats = finalstats.assign(FBP = finalstats['FB'] / finalstats['G'])
finalstats = finalstats.assign(WP = finalstats['W'] / finalstats['G'])

Ok, now the penultimate step.  We're looking for the historically best player, so we want to consider only those players who have had a meaningfully long career.  That means we're going to exclude from consideration anyone whose number of games played is in the bottom 50th percentile.  From looking at the stats this eliminates anyone who's played fewer than 65 games.  Apologies to those players, but you just didn't make the cut this time.

In [14]:
finalstats = finalstats[finalstats['G'] >= finalstats.quantile(0.5)['G']]

In [15]:
finalstats

,,M,G,W,L,K,D,A,FB,KDA,KPP,KSP,CSM,GLDP,DPM,DTHP,WPM,WCPM,FBP,WP
Player,POS,,,,,,,,,,,,,,,,,,,
369,TOP,91,229,153,76,676,571,1321,15,3.531878,57.387336,19.496070,8.151092,21.019214,465.672926,21.627948,0.371921,0.200000,0.065502,0.668122
957,TOP,78,198,108,90,351,383,1148,7,3.900000,61.500000,14.400000,8.100000,20.200000,333.900000,17.800000,0.480000,0.110000,0.035354,0.545455
ADD,TOP,143,339,146,193,621,958,1649,28,2.365487,63.752507,17.237463,7.823009,20.823009,452.532153,23.882006,0.603451,0.204749,0.082596,0.430678
AKi,JNG,35,82,34,48,193,206,497,6,3.312195,70.129268,19.765854,5.853659,17.975610,286.256098,17.617073,0.340000,0.339512,0.073171,0.414634
Adryh,ADC,59,72,26,46,245,208,343,6,2.800000,68.100000,28.400000,8.000000,23.300000,538.700000,17.100000,0.210000,0.160000,0.083333,0.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ppgod,SUP,48,122,65,57,58,370,1015,3,2.871311,68.936066,3.756557,1.157377,12.870492,132.998361,24.356557,1.456967,0.321393,0.024590,0.532787
pyl,SUP,87,215,90,125,178,666,1641,9,2.700000,67.900000,6.600000,1.500000,14.500000,150.500000,21.900000,1.340000,0.370000,0.041860,0.418605
sOAZ,TOP,149,251,146,105,531,647,1413,21,3.023904,61.651793,16.914741,7.424701,20.244223,386.234263,23.872510,0.490279,0.131912,0.083665,0.581673


## The Dessert

Onto the judging.  I choose to use a simple formula to evaluate the players, in essence a weighted sum.  For each stat I want to include, I will divide a player's stat by the maximum value for that stat amongst all players and multiply it by a weight chosen by me.  Summing up the values for each included stat will then yield an effective score for each player, and the player with the highest score is the best.

I have three categories for weights:
High Value (10 pts)
Medium Value (7 pts)
Low Value (3 pts)

The stats I will include are all of the 'averaged' stats (KDA, KPP, KSP, ... , FBP, WP).  Each role (except for Mid and Top) will have a different arrangement of weights assigned to each stat.  There is one notable exception, and that is for DTHP.  Fewer deaths is better, so instead of adding the weighted value of the DTHP stat we will subtract it.

### Top

High Value : KDA, KPP, FBP  
Medium Value : CSM, GLDP, DPM, DTHP, WP  
Low Value : WPM, WCPM

In [16]:
top = finalstats.reset_index()[finalstats.reset_index()['POS'] == 'TOP']
top = top.set_index(['Player'])
top = top.drop(labels=['POS'], axis=1)

topranking = top.assign(Rank = (top['KDA'] / top['KDA'].max()) * 10
                              + (top['KPP'] / top['KPP'].max()) * 10
                              + (top['KSP'] / top['KSP'].max()) * 7
                              + (top['CSM'] / top['CSM'].max()) * 7
                              + (top['GLDP'] / top['GLDP'].max()) * 7
                              + (top['DPM'] / top['DPM'].max()) * 7
                              + (top['DTHP'] / top['DTHP'].max()) * (-7)
                              + (top['WPM'] / top['WPM'].max()) * 3
                              + (top['WCPM'] / top['WCPM'].max()) * 3
                              + (top['WP'] / top['WP'].max()) * 7
                              + (top['FBP'] / top['FBP'].max()) * 10).iloc[:,19].sort_values(ascending=False)

topranking

Player
Knight         58.184081
Doinb          56.386889
Chovy          52.285173
Xiaohu         51.340750
BrokenBlade    49.805394
                 ...    
Chelizi        35.763269
Hoya           35.576913
Lies           34.576357
Aodi           34.219999
Max            31.193687
Name: Rank, Length: 88, dtype: float64

### Jungle

High Value : KPP, FBP, WP  
Medium Value : KDA, DTHP, WPM, WCPM, DPM  
Low Value : KSP, CSM, GLDP

In [17]:
jungle = finalstats.reset_index()[finalstats.reset_index()['POS'] == 'JNG']
jungle = jungle.set_index(['Player'])
jungle = jungle.drop(labels=['POS'], axis=1)

jungleranking = jungle.assign(Rank = (jungle['KDA'] / jungle['KDA'].max()) * 7
                              + (jungle['KPP'] / jungle['KPP'].max()) * 10
                              + (jungle['KSP'] / jungle['KSP'].max()) * 3
                              + (jungle['CSM'] / jungle['CSM'].max()) * 3
                              + (jungle['GLDP'] / jungle['GLDP'].max()) * 3
                              + (jungle['DPM'] / jungle['DPM'].max()) * 7
                              + (jungle['DTHP'] / jungle['DTHP'].max()) * (-7)
                              + (jungle['WPM'] / jungle['WPM'].max()) * 7
                              + (jungle['WCPM'] / jungle['WCPM'].max()) * 7
                              + (jungle['WP'] / jungle['WP'].max()) * 10
                              + (jungle['FBP'] / jungle['FBP'].max()) * 10).iloc[:,19].sort_values(ascending=False)

jungleranking

Player
Kanavi       47.741098
Jankos       47.130745
Score        46.875908
Blaber       45.988943
Jiejie       45.797455
               ...    
World6       33.872606
Griffin      33.113357
Swift        33.102936
AKi          32.878432
Chieftain    28.770625
Name: Rank, Length: 80, dtype: float64

### Mid

High Value : KDA, KPP, FBP  
Medium Value : CSM, GLDP, DPM, DTHP, WP  
Low Value : WPM, WCPM

In [18]:
mid = finalstats.reset_index()[finalstats.reset_index()['POS'] == 'MID']
mid = mid.set_index(['Player'])
mid = mid.drop(labels=['POS'], axis=1)

midranking = mid.assign(Rank = (mid['KDA'] / mid['KDA'].max()) * 10
                              + (mid['KPP'] / mid['KPP'].max()) * 10
                              + (mid['KSP'] / mid['KSP'].max()) * 7
                              + (mid['CSM'] / mid['CSM'].max()) * 7
                              + (mid['GLDP'] / mid['GLDP'].max()) * 7
                              + (mid['DPM'] / mid['DPM'].max()) * 7
                              + (mid['DTHP'] / mid['DTHP'].max()) * (-7)
                              + (mid['WPM'] / mid['WPM'].max()) * 3
                              + (mid['WCPM'] / mid['WCPM'].max()) * 3
                              + (mid['WP'] / mid['WP'].max()) * 7
                              + (mid['FBP'] / mid['FBP'].max()) * 10).iloc[:,19].sort_values(ascending=False)

midranking

Player
Knight       57.539568
Nisqy        56.889864
Caps         56.695282
Chovy        56.319609
ShowMaker    56.117573
               ...    
Yuuki        42.466293
SOLKA        42.297861
Damonte      42.230309
Forge        41.139834
Uniboy       37.557954
Name: Rank, Length: 68, dtype: float64

### ADC

High Value : KDA, KPP, DPM, DTHP  
Medium Value : KSP, CSM, GLDP, WP  
Low Value : WPM, WCPM, FBP

In [19]:
adc = finalstats.reset_index()[finalstats.reset_index()['POS'] == 'ADC']
adc = adc.set_index(['Player'])
adc = adc.drop(labels=['POS'], axis=1)

adcranking = adc.assign(Rank = (adc['KDA'] / adc['KDA'].max()) * 10
                              + (adc['KPP'] / adc['KPP'].max()) * 10
                              + (adc['KSP'] / adc['KSP'].max()) * 7
                              + (adc['CSM'] / adc['CSM'].max()) * 7
                              + (adc['GLDP'] / adc['GLDP'].max()) * 7
                              + (adc['DPM'] / adc['DPM'].max()) * 10
                              + (adc['DTHP'] / adc['DTHP'].max()) * (-10)
                              + (adc['WPM'] / adc['WPM'].max()) * 3
                              + (adc['WCPM'] / adc['WCPM'].max()) * 3
                              + (adc['WP'] / adc['WP'].max()) * 7
                              + (adc['FBP'] / adc['FBP'].max()) * 3).iloc[:,19].sort_values(ascending=False)

adcranking

Player
Rekkles    54.289833
Upset      51.430376
Attila     50.890959
Teddy      50.720388
Uzi        50.442218
             ...    
HeaQ       39.318726
BAO        38.411581
Adryh      37.659990
QiuQiu     36.019953
Tabzz      34.434309
Name: Rank, Length: 68, dtype: float64

### Support

High Value : KPP, FBP, WP  
Medium Value : KDA, DTHP, WPM, WCPM  
Low Value : KSP, CSM, GLDP, DPM  

In [20]:
support = finalstats.reset_index()[finalstats.reset_index()['POS'] == 'SUP']
support = support.set_index(['Player'])
support = support.drop(labels=['POS'], axis=1)

supportranking = support.assign(Rank = (support['KDA'] / support['KDA'].max()) * 7
                                + (support['KPP'] / support['KPP'].max()) * 10
                                + (support['KSP'] / support['KSP'].max()) * 3
                                + (support['CSM'] / support['CSM'].max()) * 3
                                + (support['GLDP'] / support['GLDP'].max()) * 3
                                + (support['DPM'] / support['DPM'].max()) * 3
                                + (support['DTHP'] / support['DTHP'].max()) * (-7)
                                + (support['WPM'] / support['WPM'].max()) * 7
                                + (support['WCPM'] / support['WCPM'].max()) * 7
                                + (support['WP'] / support['WP'].max()) * 10
                                + (support['FBP'] / support['FBP'].max()) * 10).iloc[:,19].sort_values(ascending=False)

supportranking

Player
Spirit        44.067506
SofM          40.908513
Lwx           38.831414
JackeyLove    38.357883
WildTurtle    37.869742
                ...    
Secret        26.155150
Ley           26.124280
Duan          25.767448
Cat           23.187557
Maestro       22.488464
Name: Rank, Length: 87, dtype: float64

So there you have it.  Knight is the best top and mid laner, Kanavi is the best jungler, Rekkles the best ADC and Spirit the best support.

## Nightcap, or how all of this should be taken with a mountain of salt

This is primarily for my own benefit and improving my data science skills.  While I am a longtime spectator of professional League of Legends play, only a small portion of my effort was used to determine a ranking formula for players.  Getting a reasonable estimate for the best player in each role should go much further than the basic statistical methods that I have applied quite loosely in this context.  My goal is to post this somewhat quickly rather than pouring several weeks to months of my life into refining my methodology to approach a more robust answer, so some quality will be sacrificed.  That being said, let's dive into the cracks at the foundation of this work.

For one, I have no idea about the legitimacy of the stats used.  I found nothing on the rotowire site that indicated how these stats were gathered.  They do appear to be a fantasy betting site which would lend credence to the stats, but that's not a compelling enough reason on its own.

For another, the stats themselves are very limited in what they look at.  Nothing is said about taking towers/inhibitors or neutral objectives, so those contributions aren't accounted for.  Also lacking is any mention of titles/tournaments won.

On my end, the whole formula for determining the best player is a prime target for criticism.  Why is a linear sum the best way to represent a player's overall value?  Why should First Blood percentage be valued more highly for a support than for an ADC?  

Finally, the use of stats in this way kind of dubious from the outset.  League of Legends is such a complicated and ever-changing game that there is a serious question of how stats from a past year can be combined with stats from later years.  The game undergoes massive changes, from how towers work (health, gold value) to how neutral objectives in the jungle operate (elemental drakes vs gold-giving dragons) to adding new champions to the game.  Even the meta shifts how the game is played.  How do stats from the lane-swap meta measure up against stats from the more recent fasting-Senna meta?  Both of these produce large changes in how the game is played on a fundamental level, so the validity of combining stats from both these eras to produce one single number is far from settled.  Beyone that, each game is vastly different from the one before it.  Baseball is a nice sport for this kind of statistical outlook because it consists of many instances of largely the same events.  A player stepping up to bat against the New York Yankees is largely the same player when stepping up to bat against the Boston Red Sox.  But the jungler who picks Sejuani against SKT plays and operates very differently than when they pick Nidalee against RNG.  Even the separate and largely isolated regional play challenges the idea of comparing all players in a role against each other.    